* Полезные ссылки<br>
  * [Web Scraping](https://coderlessons.com/tutorials/python-technologies/izuchite-python-web-scraping/python-web-scraping-kratkoe-rukovodstvo)<br>
  * [Python - Regular Expressions](https://www.tutorialspoint.com/python/python_reg_expressions.htm)<br>
  * [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)<br>
  * [AWS Amazon](https://aws.amazon.com/ru/getting-started/)<br> 
  * [Список парсеров](https://habr.com/ru/company/click/blog/494020/)<br> 
  * [Конкурентный анализ](https://blog.click.ru/semantics/kak-besplatno-sobrat-klyuchevye-slova-i-obyavleniya-konkurentov/)
  * [ВКшка](https://vc.ru/marketing/138412-instrukciya-kak-ispolzovat-parsing-vkontakte)
  * [Soup](https://pythonru.com/biblioteki/parsing-na-python-s-beautiful-soup)
  * [Scrapy](https://pythonru.com/biblioteki/sozdanie-parserov-s-pomoshhju-scrapy-i-python)
  * [Библиотека Супа](https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/)
  * [Парсинг Хабр](https://habr.com/ru/post/504900/)
  
«Inspect» (CTRL+SHIFT+I)

In [1]:
import requests as r
import lxml
from lxml import html 
import urllib3
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
import boto3
from urllib.request import urlopen
import datetime
import random
import sqlite3
import pymysql
import os
import webbrowser
import json
from urllib.request import urlopen

In [ ]:
# Скарпинг элементов с одной страницы
    
url = 'https://quotes.toscrape.com/'
response = r.get(url)
soup = BeautifulSoup(response.text, 'lxml')
quotes = soup.find_all('span', class_='text')
authors = soup.find_all('small', class_='author')
tags = soup.find_all('div', class_='tags')

for i in range(0, len(quotes)):
    print(quotes[i].text)
    print('--' + authors[i].text)
    tagsforquote = tags[i].find_all('a', class_='tag')
    for tagforquote in tagsforquote:
        print(tagforquote.text)
    print('\n')
    
url = 'https://scrapingclub.com/exercise/list_basic/?page=1'
response = r.get(url)
soup = BeautifulSoup(response.text, 'lxml')
items = soup.find_all('div', class_='col-lg-4 col-md-6 mb-4')

for n, i in enumerate(items, start=1):
    itemName = i.find('h4', class_='card-title').text.strip()
    itemPrice = i.find('h5').text
    print(f'{n}:  {itemPrice} за {itemName}')

In [ ]:
# Скарпинг элементов с нескольких страниц
url = 'https://scrapingclub.com/exercise/list_basic/?page=1'
response = r.get(url)
soup = BeautifulSoup(response.text, 'lxml')
items = soup.find_all('div', class_='col-lg-4 col-md-6 mb-4')

for n, i in enumerate(items, start=1):
    itemName = i.find('h4', class_='card-title').text.strip()
    itemPrice = i.find('h5').text
    print(f'{n}:  {itemPrice} за {itemName}')

# Ищем адреса страниц и собираем их в список. UL - галвный, LI - указатели на список    
pages = soup.find('ul', class_='pagination')
urls = []
links = pages.find_all('a', class_='page-link')

# Проверяем валидность полученных адресов
for link in links:
    pageNum = int(link.text) if link.text.isdigit() else None
    if pageNum != None:
        hrefval = link.get('href')
        urls.append(hrefval)

# Проходим по списку адресов и забираем по каждому из них нужные элементы        
for slug in urls:
    newUrl = url.replace('?page=1', slug)
    response = r.get(newUrl)
    soup = BeautifulSoup(response.text, 'lxml')
    items = soup.find_all('div', class_='col-lg-4 col-md-6 mb-4')
    for n, i in enumerate(items, start=n):
        itemName = i.find('h4', class_='card-title').text.strip()
        itemPrice = i.find('h5').text
        print(f'{n}:  {itemPrice} за {itemName}')
        
# Оптимизированный аналог

url = 'https://scrapingclub.com/exercise/list_basic/'
params = {'page': 1}
# задаем число больше номера первой страницы, для старта цикла
pages = 2
n = 1

while params['page'] <= pages:
    response = r.get(url, params=params)
    soup = BeautifulSoup(response.text, 'lxml')
    items = soup.find_all('div', class_='col-lg-4 col-md-6 mb-4')

    for n, i in enumerate(items, start=n):
        itemName = i.find('h4', class_='card-title').text.strip()
        itemPrice = i.find('h5').text
        print(f'{n}:  {itemPrice} за {itemName}')

    # [-2] предпоследнее значение, потому что последнее "Next"
    last_page_num = int(soup.find_all('a', class_='page-link')[-2].text)
    pages = last_page_num if pages < last_page_num else pages
    params['page'] += 1


In [ ]:
#Объект ответа в HTTP. Получаем инфу о контенте 
url = "https://authoraditiagarwal.com/wpcontent/uploads/2018/05/MetaSlider_ThinkBig-1080x180.jpg"

r = r.get(url, allow_redirects=True) #запрос для выполнения HTTP-запросов GET для URL
for headers in r.headers: 
    print(headers)
    
#загружаем контент

r = r.get(url) 
with open("ThinkBig.png",'wb') as f:
   f.write(r.content)

print (r.headers.get('content-type'))
print (r.headers.get('Server'))

In [ ]:
#запишем захваченные данные в файл CSV с именем dataprocessing.csv в этой папке
f = csv.writer(open(' dataprocessing.csv ','w'))
f.writerow(['Title'])
f.writerow([soup.title.text])

In [ ]:
#scraping Wikipedia to find out all the countries in Asia.


website_url = r.get('https://en.wikipedia.org/wiki/List_of_Asian_countries_by_area').text
soup = BeautifulSoup(website_url, 'lxml')
My_table = soup.find('table',{'class': 'wikitable sortable'})
links = My_table.find_all("a")
Countries = []
for link in links:
    Countries.append(link.get('title'))

df = pd.DataFrame()
df["Country"] = Countries

df

In [ ]:
# для хранения данных в корзину S3 нам нужно создать клиент S3 
s3 = boto3.client('s3')
bucket_name = "our-content"
#Созбдаем сегмент S3
s3.create_bucket(Bucket = bucket_name, ACL = 'public-read')
s3.put_object(Bucket = bucket_name, Key = '', Body = data, ACL = "public-read")

In [ ]:
#Подключаемся к SQL серверу

conn = pymysql.connect(host='127.0.0.1',user='root', passwd = None, db = 'mysql',
charset = 'utf8', port=None)
cur = conn.cursor()
cur.execute("USE scrap")
random.seed(datetime.datetime.now())
def store(title, content):
   cur.execute('INSERT INTO scrap_pages (title, content) VALUES ''("%s","%s")', (title, content))
   cur.connection.commit()

In [ ]:
#Запускаем ХромДрайвер
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.get('http://www.google.com/');
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
driver.quit()

In [ ]:
# Извлечение веб данных о системе пользователя
import user_agents
ua = user_agents.parse(ua)
ua.is_bot
ua.is_mobile
ua.os.family
ua.browser.family

In [ ]:
#robots.txt — это файл, используемый для идентификации частей сайта, которые сканерам разрешено просматривать 
#Sitemap - карты сайта, которые помогают сканерам находить контент без необходимости сканировать каждую страницу